In [1]:
# !pip install toytree

     |████████████████████████████████| 108 kB 508 kB/s eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
     |████████████████████████████████| 269 kB 669 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 172 kB/s eta 0:00:01
     |████████████████████████████████| 58 kB 446 kB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 1.5 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=6a9081155a40034ced1fc1a16e028ec88f305cbf6c9c43486d070590e851cdea
  Stored in directory: /home/srmq/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
  Created wheel for toyplot: filename=toyplot-1.0.1-py3-none-any.whl size=283807 sha256=9fd372eb8e7ce45b0e4698a5eb6efb697843df9c078901b9a99e1ed98d559144
  Stored in directory: /home/srmq/.cache/pip/wheels/e7/ba/fe/920b8c77e332e2cc8faf82a9325162f436786656a85d69afd2
Successfully built future toyplot


In [2]:
import toytree       # a tree plotting library
import toyplot       # a general plotting library
import numpy as np   # numerical library

# Auto Rebalance Portfolio

In [3]:
newick = "((bitcoin)40:3,(((ethereum)5, (bsc)5, (polygon)5, (avalanche)5, (moonriver)5, (cex)5)30,((defi)6, (gaming)6, (nft)6, (meme)6, (other)6)30:2)60:1)100;"
tre0 = toytree.tree(newick, tree_format=0)

In [4]:
# show support values
tre0.draw(
    node_labels=tre0.get_node_values("support", 1, 0),
    node_sizes=20,
    );

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="295.0px" height="275.0px" viewBox="0 0 295.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5f837419d631417c83bbe7011943e5c7"> 5 5 5 5 5 5 6 6 6 6 6 30 30 60 40 100 cex moonriver avalanche polygon bsc ethereum other meme nft gaming defi bitcoin

Our auto rebalance portfolio will be initially organized as above.

`CREATE TABLE onn_daily(indexId SYMBOL capacity 64, isoInstant TIMESTAMP, indexValue DOUBLE, isRebalanceDay BOOLEAN, divisor DOUBLE, lastRebalanceISOInstant TIMESTAMP, indexConstituents STRING), index(indexId) timestamp(isoInstant) PARTITION BY YEAR;`

onn_daily2 table will use the 2nd form (as below)

In [6]:
newick = "((bitcoin)40:3,(((smart-contract-platform)25, (cex)5)30,((defi)6, (gaming)6, (nft)6, (meme)6, (other)6)30:2)60:1)100;"
tre0 = toytree.tree(newick, tree_format=0)
tre0.draw(
    node_labels=tre0.get_node_values("support", 1, 0),
    node_sizes=20,
    );

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="365.0px" height="275.0px" viewBox="0 0 365.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tcfde1782629f4996ba20746e806ea3ac"> 6 6 6 6 6 5 25 30 30 60 40 100 other meme nft gaming defi cex smart-contract-platform bitcoin

`CREATE TABLE onn_daily2(indexId SYMBOL capacity 64, isoInstant TIMESTAMP, indexValue DOUBLE, isRebalanceDay BOOLEAN, divisor DOUBLE, lastRebalanceISOInstant TIMESTAMP, indexConstituents STRING), index(indexId) timestamp(isoInstant) PARTITION BY YEAR;`

In [8]:
import glob
import json
import httpx
jsonFiles = glob.glob('ONN*.json')
host = 'http://localhost:9000'

async def executeQuery(valStrings):
    sql_query = "INSERT INTO onn_daily2 VALUES"
    sql_query += valStrings[0]
    for i in range(1, len(valStrings)):
        sql_query += f",{valStrings[i]}"
    query_params = {'query': sql_query, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
        await client.get(host + '/exec', params=query_params)

lotSize=5
for fileName in jsonFiles:
    print (f"Inserting {fileName}...")
    with open(fileName, 'r') as f:
        indexData = json.loads(f.read())
        valStrings = []
        for indexPt in indexData:
            if len(valStrings) == lotSize:
                await executeQuery(valStrings)
                valStrings = []
            val = f"(\'{indexPt['indexId']}\', \'{indexPt['isoInstant']}\', {indexPt['indexValue']}, {indexPt['isRebalanceDay']}, {indexPt['divisor']}, \'{indexPt['lastRebalanceISOInstant']}\', \'{json.dumps(indexPt['indexConstituents'])}\')"
            valStrings.append(val) 

Inserting ONN-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-Alt-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-Vap-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(smart-contract-platform)-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(centralized-exchange-token-cex)-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(decentralized-finance-defi)-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(non-fungible-tokens-nft)-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(meme-token)-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(gaming)-B-30dW-30dR_2020-12-01_2022-07-01.json...
Inserting ONN-(Other)-B-30dW-30dR_2020-12-01_2022-07-01.json...


# Now let's simulate auto-rebalance in categories

In [7]:
categories = {
    'bitcoin':0.4, 
    'decentralized-finance-defi':0.06, 
    'gaming':0.06, 
    'non-fungible-tokens-nft':0.06, 
    'meme-token':0.06, 
    'Other':0.06,
    'ethereum-ecosystem': 0.05,
    'binance-smart-chain': 0.05,
    'polygon-ecosystem': 0.05,
    'avalanche-ecosystem': 0.05,
    'moonriver-ecosystem': 0.05,
    'centralized-exchange-token-cex': 0.05           
} # this was the first version

In [9]:
categories = {
    'bitcoin':0.4, 
    'decentralized-finance-defi':0.06, 
    'gaming':0.06, 
    'non-fungible-tokens-nft':0.06, 
    'meme-token':0.06, 
    'Other':0.06,
    'smart-contract-platform': 0.25,
    'centralized-exchange-token-cex': 0.05           
}
totalPerc = 0.0
for p in categories:
    totalPerc += categories[p]
assert(abs(totalPerc - 1.0) < 0.0000001)
del totalPerc

In [10]:
indexIds = []
for categ in categories:
    if categ != 'bitcoin':
        indexIds.append(f"ONN-({categ})-B-30dW-30dR")
indexIds

['ONN-(decentralized-finance-defi)-B-30dW-30dR',
 'ONN-(gaming)-B-30dW-30dR',
 'ONN-(non-fungible-tokens-nft)-B-30dW-30dR',
 'ONN-(meme-token)-B-30dW-30dR',
 'ONN-(Other)-B-30dW-30dR',
 'ONN-(smart-contract-platform)-B-30dW-30dR',
 'ONN-(centralized-exchange-token-cex)-B-30dW-30dR']

In [11]:
categories

{'bitcoin': 0.4,
 'decentralized-finance-defi': 0.06,
 'gaming': 0.06,
 'non-fungible-tokens-nft': 0.06,
 'meme-token': 0.06,
 'Other': 0.06,
 'smart-contract-platform': 0.25,
 'centralized-exchange-token-cex': 0.05}

In [12]:
import pandas as pd
btcDF = pd.read_csv('Historical-BTCUSDT-Binance.csv', parse_dates = ['open_time', 'close_time'], infer_datetime_format=True)
btcDF['strDate'] = btcDF['open_time'].dt.strftime('%Y-%m-%d')
btcDF.set_index('strDate', drop=True, inplace=True)
btcDF.rename(columns={'close':'BTC'}, inplace=True)
dfBTCIndex = btcDF[['BTC']].sort_values('strDate')
dfBTCIndex['BTC'] = 1000*(dfBTCIndex['BTC']/(dfBTCIndex['BTC'][0]))
del btcDF
display(dfBTCIndex)


,BTC
strDate,
2020-12-01,1000.000000
2020-12-02,1023.401595
2020-12-03,1035.008868
2020-12-04,993.901399
2020-12-05,1020.394395
...,...
2022-06-20,1096.399353
2022-06-21,1104.373257
2022-06-22,1065.176265


In [13]:
from typing import Dict
from dateutil.parser import parse
import datetime

#falta levar em consideração o perdido mensalmente com rebalance dos índices, mas isso sera feito so quando for comprar as assets subjacentes
async def simulateDailyRebal(startValue: float, isoInstantStart: str, isoInstantEnd: str, 
                             categWeights: Dict[str, float], discountSell: 
                             float, discountBuy: float) -> Dict[str, object]:
    wallet = {}
    wallet['portfolio'] = {}
    wallet['totalValue'] = 0.0
    wallet['pocketMoney'] = 0.0
    categNames = list(categWeights.keys())

    async def categIndexVals(categNames: [str], isoInstant) -> Dict[str, float]:
        categIndexIds = categs2IndexIds(categNames)
        indices = await getIndices(str(isoInstant), categNames)
        result = {}
        for cat in categNames:
            if cat == 'bitcoin':
                indexVal = float(dfBTCIndex.loc[isoInstant[:10]])
            else:
                indexId = categIndexIds[cat]
                indices = await getIndices(isoInstant, categNames)
                indexData = indices[indexId]
                indexVal = indexData.indexValue
            result[cat] = indexVal
        return result
    lostToRebal = startValue*discountBuy
    startIndices = await categIndexVals(categNames, isoInstantStart)
    for cat in categWeights:
        buyCat = categWeights[cat]*startValue
        buyCat *= 1.0 - discountSell
        wallet['portfolio'][cat] = buyCat/startIndices[cat]
        wallet['totalValue'] += buyCat
    result = {}
    result['days'] = {}
    result['days'][isoInstantStart] = wallet
    print(f"{isoInstantStart},{wallet}")
    currentDay = parse(isoInstantStart) + datetime.timedelta(days = 1)
    while (currentDay < parse(isoInstantEnd)):
        currIndices = await categIndexVals(categNames, str(currentDay))
        currWalletVals = {}
        totalValue = 0.0
        for cat in wallet['portfolio']:
            currWalletVals[cat] = wallet['portfolio'][cat]*currIndices[cat]
            totalValue += currWalletVals[cat]
        pocketMoney = 0.0
        if 'pocketMoney' in wallet:
            pocketMoney += wallet['pocketMoney']
        #selling excess
        excessCats = set()
        for cat in wallet['portfolio']:
            excessWeight = (currWalletVals[cat]/(totalValue+pocketMoney)) - categWeights[cat]
            if excessWeight > 0:
                excessCats.add(cat)
                sellAmount = excessWeight*(totalValue+pocketMoney)
                pocketMoney += sellAmount*(1.0 - discountSell)
                currWalletVals[cat] -= sellAmount
                lostToRebal += sellAmount*(discountSell) 
                totalValue -= sellAmount*(discountSell)
        notExcessCats = set(wallet['portfolio'].keys()) - excessCats
        for cat in notExcessCats:
            missingWeight = categWeights[cat] - (currWalletVals[cat]/(totalValue+pocketMoney))
            if missingWeight > 0:
                buyAmount = missingWeight*(totalValue+pocketMoney)
                if pocketMoney < buyAmount:
                    print(f"Warning: {pocketMoney} of pocket money cannot buy {buyAmount}, ignoring")
                else:
                    currWalletVals[cat] += buyAmount*(1.0 - discountBuy)
                    pocketMoney -= buyAmount
                    lostToRebal += buyAmount*discountBuy
                    totalValue -= buyAmount*discountBuy
        newWallet = {}
        newWallet['totalValue'] = totalValue
        newWallet['pocketMoney'] = pocketMoney
        newWallet['portfolio'] = {}
        for cat in categWeights:
            newWallet['portfolio'][cat] = currWalletVals[cat]/currIndices[cat]
        wallet = newWallet
        result['days'][str(currentDay)] = wallet
        print(f"{currentDay},{wallet}")
        currentDay += datetime.timedelta(days = 1)
    result['totalLostToRebal'] = lostToRebal
    return result

In [14]:
def categs2IndexIds(categs: [str]) -> Dict[str, str]:
    result = {}
    for categ in categs:
        if categ != 'bitcoin':
            result[categ] = f"ONN-({categ})-B-30dW-30dR"
    return result

In [15]:
import json
import pandas as pd
class IndexData:
    indexId: str
    _indexValue: float
    isoInstant: str
    isRebalanceDay: bool
    _divisor: float
    lastRebalanceISOInstant: str
    indexConstituents: pd.DataFrame
    
    class IndexDataEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, IndexData):
                return {"indexId": obj.indexId, "isoInstant": obj.isoInstant, "indexValue": obj.indexValue, "isRebalanceDay": obj.isRebalanceDay, "divisor": obj.divisor, "lastRebalanceISOInstant": obj.lastRebalanceISOInstant, "indexConstituents": json.loads(obj.indexConstituents.to_json(orient='split', index=False))}
            return json.JSONEncoder.default(self, obj)
    
    def set_indexValue(self, value):
        self._indexValue = round(value, 2)
    
    def get_indexValue(self):
        return self._indexValue
    
    def del_indexValue(self):
        del self._indexValue
        
    indexValue = property(get_indexValue, set_indexValue, del_indexValue)
    
    def set_divisor(self, value):
        self._divisor = round(value, 4)
    
    def get_divisor(self):
        return self._divisor
    
    def del_divisor(self):
        del self._divisor
       
    divisor = property(get_divisor, set_divisor, del_divisor)
    
    def toJSON(self):
        return json.dumps(self, cls=IndexData.IndexDataEncoder)
    
    def __repr__(self):
        return self.toJSON()
    
    def __jsonDecode(dct):
        attrs = ["indexId", "isoInstant", "indexValue", "isRebalanceDay", "divisor", "lastRebalanceISOInstant", "indexConstituents"]
        for attr in attrs:
            if attr not in dct:
                return dct
        result = IndexData()
        result.indexId = dct['indexId']
        result.isoInstant = dct['isoInstant']
        result.indexValue = dct['indexValue']
        result.isRebalanceDay = dct['isRebalanceDay']
        result.divisor = dct['divisor']
        result.lastRebalanceISOInstant = dct['lastRebalanceISOInstant']
        result.indexConstituents = pd.read_json(json.dumps(dct['indexConstituents']), orient='split')
        return result
    
    def fromJSON(jsStr):
        return json.loads(jsStr, object_hook=IndexData.__jsonDecode)

In [16]:
import pandas as pd
from typing import Dict
import httpx
host = 'http://localhost:9000'
async def getIndices(isoInstant: str, catNames: [str]) -> Dict[str, IndexData]:
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result    
    
    columns = ['indexId', 'isoInstant', 'indexValue', 'isRebalanceDay', 'divisor', 'lastRebalanceISOInstant', 'indexConstituents']
    indexIds = list(categs2IndexIds(catNames).values())
    sqlIn = f"\'{indexIds[0]}\'"
    for i in range(1, len(indexIds)):
        sqlIn += f", \'{indexIds[i]}\'"
    sqlIn = "(" + sqlIn + ")"
    sqlStmt = "SELECT %s FROM onn_daily2 WHERE isoInstant = '%s' AND indexId IN %s"%(listToStr(columns), isoInstant, sqlIn)
    # print(sqlStmt)
    query_params = {'query': sqlStmt, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
        r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    # print(jsonR)
    resultArray = pd.DataFrame(columns=columns, data=jsonR['dataset'])
    resultArray = resultArray.to_dict('records')
    result = {}
    for rec in resultArray:
        rec['indexConstituents'] = json.loads(rec['indexConstituents'])
        result[rec['indexId']] = IndexData.fromJSON(json.dumps(rec))
    return result

In [17]:
result = await simulateDailyRebal(1000, '2020-12-01T00:00:00.000000Z', "2022-06-25T00:00:00.000000Z", categories, 0.001, 0.001)

2020-12-01T00:00:00.000000Z,{'portfolio': {'bitcoin': 0.3996, 'decentralized-finance-defi': 0.05994, 'gaming': 0.05994, 'non-fungible-tokens-nft': 0.05994, 'meme-token': 0.05994, 'Other': 0.05994, 'smart-contract-platform': 0.24975, 'centralized-exchange-token-cex': 0.04995}, 'totalValue': 999.0000000000002, 'pocketMoney': 0.0}
2020-12-02 00:00:00+00:00,{'totalValue': 977.8037093490499, 'pocketMoney': 4.901659205216102, 'portfolio': {'bitcoin': 0.3821899162029569, 'decentralized-finance-defi': 0.06263106105158268, 'gaming': 0.05994, 'non-fungible-tokens-nft': 0.06152955276163588, 'meme-token': 0.06360690642269236, 'Other': 0.06370699526873004, 'smart-contract-platform': 0.24975, 'centralized-exchange-token-cex': 0.05189469427965001}}
2020-12-03 00:00:00+00:00,{'totalValue': 998.126741793568, 'pocketMoney': 4.203580709546115, 'portfolio': {'bitcoin': 0.3908217070509647, 'decentralized-finance-defi': 0.058373342333287545, 'gaming': 0.05651461503849562, 'non-fungible-tokens-nft': 0.060504

In [18]:
result

{'days': {'2020-12-01T00:00:00.000000Z': {'portfolio': {'bitcoin': 0.3996,
    'decentralized-finance-defi': 0.05994,
    'gaming': 0.05994,
    'non-fungible-tokens-nft': 0.05994,
    'meme-token': 0.05994,
    'Other': 0.05994,
    'smart-contract-platform': 0.24975,
    'centralized-exchange-token-cex': 0.04995},
   'totalValue': 999.0000000000002,
   'pocketMoney': 0.0},
  '2020-12-02 00:00:00+00:00': {'totalValue': 977.8037093490499,
   'pocketMoney': 4.901659205216102,
   'portfolio': {'bitcoin': 0.3821899162029569,
    'decentralized-finance-defi': 0.06263106105158268,
    'gaming': 0.05994,
    'non-fungible-tokens-nft': 0.06152955276163588,
    'meme-token': 0.06360690642269236,
    'Other': 0.06370699526873004,
    'smart-contract-platform': 0.24975,
    'centralized-exchange-token-cex': 0.05189469427965001}},
  '2020-12-03 00:00:00+00:00': {'totalValue': 998.126741793568,
   'pocketMoney': 4.203580709546115,
   'portfolio': {'bitcoin': 0.3908217070509647,
    'decentralized-

In [19]:
import csv
with open('Historical-ONN-CategRebalDaily2-B-30dW-30dR_2020-12-01_2022-06-24.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Date', 'Close'])
    for key in result['days']:
        vals = []
        vals.append(key)
        vals.append(result['days'][key]['totalValue']+result['days'][key]['pocketMoney'])
        writer.writerow(vals)
